## ライブラリインポート

In [2]:
import numpy as np
import keras
from keras.models import  Model,load_model
from keras.layers import Dense,Dropout,Conv2D,MaxPooling2D,Input,GlobalAveragePooling2D,BatchNormalization
from keras.optimizers import Adam
# from keras.preprocessing.image import ImageDataGenerator
# from keras.callbacks import LearningRateScheduler
from keras.datasets import cifar10
from keras.utils.np_utils import to_categorical
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
import pickle

## データ整形

In [3]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [4]:
# データ次元確認
print('x_train.shape:', x_train.shape)
print('x_test.shape:', x_test.shape)
print('y_train.shape:', y_train.shape)
print('y_test.shape:', y_test.shape)

x_train.shape: (50000, 32, 32, 3)
x_test.shape: (10000, 32, 32, 3)
y_train.shape: (50000, 1)
y_test.shape: (10000, 1)


In [5]:
# 正規化
x_train = x_train/255.
x_test = x_test/255.

# one hot ベクトル化
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

## モデル作成

In [9]:
def create_bench_model():
    inputs = Input(shape = (32,32,3))
    x = Conv2D(64,(3,3),padding = "SAME",activation= "relu")(inputs)
    x = Conv2D(64,(3,3),padding = "SAME",activation= "relu")(x)
    x = Dropout(0.25)(x)
    x = MaxPooling2D()(x)

    x = Conv2D(128,(3,3),padding = "SAME",activation= "relu")(x)
    x = Conv2D(128,(3,3),padding = "SAME",activation= "relu")(x)
    x = Dropout(0.25)(x)
    x = MaxPooling2D()(x)

    x = Conv2D(256,(3,3),padding = "SAME",activation= "relu")(x)
    x = Conv2D(256,(3,3),padding = "SAME",activation= "relu")(x)
    x = GlobalAveragePooling2D()(x)

    x = Dense(1024,activation = "relu")(x)
    x = Dropout(0.25)(x)
    x = Dense(10, activation=None)(x) # 追加層
    y = Dense(10,activation = "softmax")(x)
    
    model = Model(input = inputs, output = y)

    return model

In [10]:
model = create_bench_model()
model.compile(loss = "categorical_crossentropy",optimizer = Adam(), metrics = ["accuracy"])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 32, 32, 3)         0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 32, 32, 64)        1792      
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 32, 32, 64)        36928     
_________________________________________________________________
dropout_4 (Dropout)          (None, 32, 32, 64)        0         
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 16, 16, 64)        0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 16, 16, 128)       73856     
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 16, 16, 128)       147584    
__________

c:\users\12810649\appdata\local\continuum\anaconda3\envs\tf1.1.4\lib\site-packages\ipykernel_launcher.py:22: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


In [11]:
history = model.fit(x_train,
                    y_train,
                    batch_size=32,
                    epochs=20,
                    validation_split=0.2)

Train on 40000 samples, validate on 10000 samples
Epoch 1/20
40000/40000 [==============================] - 11s 272us/step - loss: 1.7924 - acc: 0.3023 - val_loss: 1.4986 - val_acc: 0.4557
Epoch 2/20
40000/40000 [==============================] - 10s 261us/step - loss: 1.2652 - acc: 0.5378 - val_loss: 1.1801 - val_acc: 0.5697
Epoch 3/20
40000/40000 [==============================] - 10s 258us/step - loss: 0.9860 - acc: 0.6484 - val_loss: 0.9299 - val_acc: 0.6751
Epoch 4/20
40000/40000 [==============================] - 10s 261us/step - loss: 0.8253 - acc: 0.7073 - val_loss: 0.8784 - val_acc: 0.6864
Epoch 5/20
40000/40000 [==============================] - 10s 260us/step - loss: 0.7157 - acc: 0.7504 - val_loss: 0.7825 - val_acc: 0.7269
Epoch 6/20
40000/40000 [==============================] - 10s 260us/step - loss: 0.6318 - acc: 0.7811 - val_loss: 0.6908 - val_acc: 0.7577
Epoch 7/20
40000/40000 [==============================] - 10s 262us/step - loss: 0.5618 - acc: 0.8054 - val_loss: 0.

In [12]:
# Score trained model.
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

10000/10000 [==============================] - 1s 76us/step
Test loss: 0.7370336189746857
Test accuracy: 0.7983


In [13]:
import os
# Save model and weights
save_dir = os.path.join(os.getcwd(), 'models')
model_name = 'cifar10_tf114_functional_api_ver_200604.h5'
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
model.save(model_path)
print('Saved trained model at %s ' % model_path)

Saved trained model at C:\Users\12810649\Projects\dnn_samples\cifar10\models\cifar10_tf114_functional_api_ver_200604.h5 
